
# Bu soruya cevap arıyoruz: CNN’deki conv filtreleri “rastgele” mi? Filtre sırasını biz belirleyebilir miyiz? “Kenardan başlamak” mantıklı mı?

Bu notebook şu soruları teknik olarak netleştirir:

1) Convolution katmanları **rastgele filtre mi uygular**, yoksa filtreler **öğrenilir** mi?  
2) Conv katmanındaki filtrelerin **sırası** diye bir şey var mı? Biz bunu belirleyebilir miyiz?  
3) “En iyi çözüm kenardan başlamaktır” fikri nereden geliyor? CNN’ler zaten **kenar** mı öğreniyor?  
4) Eğer “kenar” gibi ön-bilgi eklemek istersek, bunu **hangi yöntemlerle** yapabiliriz?

> Not: Burada “filtre sırası” derken genelde iki şey kastedilir:
> - (A) Aynı katmandaki farklı kernel’lerin **hangi sırada/indekste** durduğu  
> - (B) Katmanların **hangi tip filtreleri önce-sonra** öğreneceği (düşük seviye → yüksek seviye)



---

## 1) Kısa cevap

- Conv filtreleri **rastgele uygulanmaz**.  
  Eğitim başında ağırlıklar **rastgele başlatılır**, ama sonra **geri yayılım (backprop)** ile veri üzerinden öğrenilir.

- Aynı katmandaki filtrelerin **sırası** genelde anlamlı değildir.  
  Çünkü ağda “kanal permütasyon simetrisi” vardır: filtreleri permüte edebilirsin, sonraki katmanın ağırlıklarını da aynı şekilde permüte edersen ağın çıktısı değişmez.

- “Kenardan başlamak” sezgisi **doğru bir induktif bias**:  
  Erken katmanlar gerçekten de sıklıkla **Gabor/edge benzeri** filtreler öğrenir.  
  Ama bunu *elle zorunlu kılmak* yerine ağın bunu öğrenmesine izin vermek çoğu zaman daha etkilidir.

Şimdi bunu temelden ispat/intuition ile açıyoruz.



---

## 2) Conv filtreleri neden “rastgele” gibi görünür?

### 2.1) Başlangıçta rastgele (initialization)

Eğitim başlamadan önce conv ağırlıkları genelde:
- He/Kaiming init (ReLU/SiLU için)
- Xavier init (tanh vb. için)

gibi dağılımlardan **rastgele** örneklenir.

Ama bu “model rastgele filtre uygular” demek değildir.  
Bu sadece başlangıç noktasını belirler.

### 2.2) Eğitimde öğrenme (learning)

Eğitim sırasında hedef fonksiyon minimize edilir:

\[
W \leftarrow W - \eta \nabla_W \mathcal{L}
\]

Bu gradyan, filtrelerin “hangi örüntüyü yakalaması gerektiği” bilgisini taşır.
Dolayısıyla filtreler zamanla veri istatistiklerine göre şekillenir.



---

## 3) “Filtre sırası” diye bir şey var mı?

### 3.1) Aynı katmandaki filtrelerin indeksi (kanal sırası) **anlamlı değil**

Bir conv katmanı \(C_{out}\) tane filtre üretir.
Bu filtrelerin “1. filtre”, “2. filtre” diye bir sırası vardır ama bu sıra **semantik** değildir.

Neden? Çünkü şu simetri vardır:

- Bir katmanın çıktı kanallarını herhangi bir permütasyonla yeniden sıralarsan,
- Sonraki katmanın giriş kanallarını aynı permütasyonla uyumlu şekilde yeniden sıralarsan,
- Ağın çıktısı **değişmez**.

Bu yüzden “filtrelerin sırası” modele bir şey katmaz; sadece bir indekslemedir.

### 3.2) Peki “katmanlar arası sıra” (low-level → high-level) var mı?

Evet, bu farklı bir şey:
- İlk katmanlar genelde kenar/texture (low-level)
- Orta katmanlar parça/şekil (mid-level)
- Derin katmanlar semantik (high-level)

Bu sıra, “indeks sırası” değil; **derinlik boyunca temsil hiyerarşisi**dir.


In [1]:

# Kanal permütasyon sezgisi: 
# Aynı katmandaki filtreleri yeniden sıralamak tek başına anlamlı değildir.
# (Matematiksel fikir: sonraki katman bu permütasyonu telafi edebilir.)

import torch

torch.manual_seed(0)

# Basit iki katmanlı conv blok (1x1 conv ile kanallar arası karışım kolay gösterilir)
conv1 = torch.nn.Conv2d(3, 8, kernel_size=3, padding=1, bias=False)
conv2 = torch.nn.Conv2d(8, 4, kernel_size=1, padding=0, bias=False)

x = torch.randn(2, 3, 16, 16)

y = conv2(conv1(x))

# Şimdi conv1'in çıkış kanallarını permüte edelim
perm = torch.randperm(8)

# conv1 ağırlıklarını permüte: out_channels yeniden sıralansın
conv1_perm = torch.nn.Conv2d(3, 8, kernel_size=3, padding=1, bias=False)
conv1_perm.weight.data = conv1.weight.data[perm].clone()

# conv2 giriş kanallarını aynı perm ile telafi edelim: conv2.weight: [out, in, 1,1]
conv2_perm = torch.nn.Conv2d(8, 4, kernel_size=1, padding=0, bias=False)
conv2_perm.weight.data = conv2.weight.data[:, perm].clone()

y2 = conv2_perm(conv1_perm(x))

print("Maks |y - y2|:", (y - y2).abs().max().item())


Maks |y - y2|: 1.1920928955078125e-07



✅ Çıktı çok küçük (sayısal hata seviyesinde) ise şu demektir:  
**Aynı katmandaki filtrelerin sırası/indeksi semantik değil**; sonraki katman bu permütasyonu telafi edebilir.

Dolayısıyla:
- “Filtrelerin sırasını belirleyeyim” fikri çoğu zaman anlamsızdır.
- Önemli olan: filtrelerin **hangi özellikleri** öğrenebildiği ve katmanların **hiyerarşik** düzenidir.



---

## 4) “Kenardan başlamak” fikri doğru mu?

### 4.1) Evet: erken katmanlar genelde edge/texture öğrenir

Doğal görüntülerde en güçlü istatistiklerden biri:
- yoğunluk değişimleri (gradient)  
- kenarlar (edges)  
- basit yönlü örüntüler (Gabor benzeri)

Bu yüzden CNN’lerin ilk katmanları sıkça:
- yatay/dikey kenar
- diyagonal kenar
- blob/spot
- düşük frekans (blur)  
benzeri filtreler öğrenir.

### 4.2) Ama “elle zorlamak” her zaman iyi değil

Eğer ilk katmanı “sadece edge filtreleri” yaparsan:
- modeli bazı veri türlerinde kısıtlarsın  
  (ör. medikal, SAR, multispectral, düşük ışık, stylized data)
- öğrenmesi gereken farklı low-level ipuçlarını kaçırabilirsin

Modern yaklaşım:
- doğru inductive bias’ı **tasarımla** ver (stride planı, norm seçimi, augmentation)
- filtrelerin detayını **öğrenmeye bırak**



---

## 5) Peki “kenarı” modele bilerek nasıl eklersin? (mantıklı yöntemler)

Bu işin 4 seviyesi var:

### 5.1) Sabit (hand-crafted) filtre katmanı
- Sobel/Scharr/Laplacian gibi kernel’leri sabit tut
- İlk katmana “ek kanal” olarak ekle (RGB + edge map)

✅ Avantaj: güçlü ön-bilgi  
❌ Dezavantaj: esneklik azalır

### 5.2) Başlangıçta edge’e benzetip sonra serbest bırakma
- İlk conv kernel’lerini Gabor benzeri başlat
- Eğitimle güncellenmesine izin ver

✅ Avantaj: iyi başlangıç  
❌ Dezavantaj: her probleme uygun değil

### 5.3) Kayıp fonksiyonuna yardımcı görev (edge supervision)
- Ek bir head ile edge map tahmini
- Multi-task learning

✅ Avantaj: representation güçlenir  
❌ Dezavantaj: ek karmaşıklık

### 5.4) Frequency-aware / anti-aliasing yaklaşımlar
- Downsample öncesi low-pass (BlurPool vb.)
- Daha sağlam sinyal işleme

✅ Avantaj: stride kaynaklı aliasing azalır  
❌ Dezavantaj: maliyet/latency etkisi olabilir


In [2]:

# Mini demo: Sobel edge çıkarıp RGB'ye ek kanal olarak birleştirme (PyTorch ile)
import torch
import torch.nn.functional as F

def sobel_edges(gray):
    # gray: (N,1,H,W)
    kx = torch.tensor([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=gray.dtype, device=gray.device).view(1,1,3,3)
    ky = torch.tensor([[-1,-2,-1],[0,0,0],[1,2,1]], dtype=gray.dtype, device=gray.device).view(1,1,3,3)
    gx = F.conv2d(gray, kx, padding=1)
    gy = F.conv2d(gray, ky, padding=1)
    mag = torch.sqrt(gx**2 + gy**2 + 1e-6)
    return mag

x = torch.randn(2, 3, 32, 32)
gray = x.mean(dim=1, keepdim=True)
edge = sobel_edges(gray)

x_plus = torch.cat([x, edge], dim=1)  # (N,4,H,W)
print("RGB:", x.shape, "RGB+edge:", x_plus.shape)


RGB: torch.Size([2, 3, 32, 32]) RGB+edge: torch.Size([2, 4, 32, 32])



Bu, “kenar bilgisini” modele **ekstra kanal** olarak vermenin en basit yolu.

Ama tekrar: bu bir “her zaman en iyi” değil; dataset’e göre değişir.



---

## 6) Sık yapılan yanlış anlama: “Filtrelerin sırasını belirleyebilirim” ≠ “Ne öğreneceğini belirlerim”

- Filtre indeksini sıralamak kolay (permütasyon)
- Ama “bu filtre kesin kenar, bu filtre kesin texture” diye zorlamak:
  - ya sabit filtre gerektirir
  - ya da ek kısıt/regularization gerektirir
  - genelde optimuma giden yolu daraltır

CNN’lerin gücü:
> Feature’ların hangi kombinasyonla faydalı olacağını veriden öğrenmesidir.



---

## 7) Kapanış: Net cevap

- Conv mimarileri “rastgele filtre uygular” gibi başlar çünkü **ağırlıklar rastgele başlatılır**.  
  Ama eğitimle filtreler **öğrenilir**.

- Aynı katmandaki filtrelerin “sırası” semantik değildir.  
  Kanal permütasyonları sonraki katman tarafından telafi edilebilir.

- “Kenardan başlamak” fikri, doğal görüntü istatistikleri yüzünden sezgisel olarak doğrudur;  
  erken katmanlar çoğu zaman edge/texture öğrenir.  
  Ama bunu “kural” gibi dayatmak yerine:
  - doğru stride/augmentation/norm seçimi
  - gerekiyorsa edge bilgisini yardımcı sinyal olarak verme
  gibi yöntemler daha güvenli.

İstersen bir sonraki adımda:
- Senin YOLO/backbone tasarımında “edge bias” eklemek mantıklı mı,  
datasetine göre nasıl karar veririz (small object/blur/noise) onu da çıkarırız.
